# Experiment 3
- Load the saved model and replace the output layer of the model, as well as the first two convolutional layers (keep the weights of all other layers).
- Train and evaluate the model on the cats and dogs dataset.

## Steps
Load the saved model from a file.

Freeze all layers except the first two convolutional layers and the output layer.

Replace the first two convolutional layers and the output layer.

Compile the model with a suitable loss function and optimizer.

Train and evaluate the model on the Cats vs. Dogs dataset.

### Load the saved model

In [ ]:
import tensorflow as tf
from tensorflow import data as tf_data
from tensorflow import keras
from keras import layers

# Load the pre-trained model
saved_model_path = "models/experiment1_model.keras"
model = keras.models.load_model(saved_model_path)

### Modify the model

In [ ]:
sizes = [256, 512, 728]

# Freeze the entire model initially
for layer in model.layers:
    layer.trainable = False

# Define new input layer
inputs = model.input

# Replace the first two convolutional layers (Match expected output filters)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)  # Match original filter size
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)  # Match next layer

# Keep the remaining layers from the original model (except first 2 conv & output layer)
for layer in model.layers[2:-1]:  # Skipping first 2 conv layers & last layer
    x = layer(x)

# Add new output layer (Binary Classification)
new_output = layers.Dense(1, activation="sigmoid")(x)  # Sigmoid for binary classification

# Create the new model
new_model = keras.Model(inputs, new_output)

# Unfreeze only the modified first two convolutional layers & the output layer
for layer in new_model.layers[:3]:  # First 2 conv layers + input layer
    layer.trainable = True

# Print model summary
new_model.summary()


### Compile the model

In [ ]:
new_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)


### Create the dataset

In [ ]:
# Create the dataset
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x), y))

# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

### Train and evaluate the model

In [ ]:
epochs = 50
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="models/experiment3_epoch_{epoch}.keras", period=10)
]

new_model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=callbacks
)

new_model.save("models/experiment4_model.keras")
